<a href="https://colab.research.google.com/github/Diame58/Pronobo/blob/main/probono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import librosa
import librosa.display

import plotly.express as px
import IPython.display as ipd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from tqdm import tqdm, trange
from librosa import feature, amplitude_to_db, load

from tqdm.auto import tqdm
from plotly.subplots import make_subplots

from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout

pd.plotting.register_matplotlib_converters()

%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
data_path = '/content/drive/MyDrive/UrbanSound8K.csv'
df = pd.read_csv(data_path)
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [38]:
sample_path = []
path = '/content/drive/MyDrive/'

for index_num,row in tqdm(df.iterrows()):
    file_name = os.path.join(os.path.abspath(path),'fold'+str(row["fold"])+'/',str(row['slice_file_name']))
    sample_path.append(file_name)
df['path'] = sample_path

0it [00:00, ?it/s]

In [39]:
count1 = (df['class'] == "car_horn").sum()
count3 = (df['class'] == "dog_bark").sum()
count8 = (df['class'] == "siren").sum()
print(count1,count3,count8)
#클래스1,3,8인 경우의 데이터 개수

429 1000 929


In [40]:
target_count = count1

car_horn_data = df[df['class'] == "car_horn"]
dog_bark_data = df[df['class'] == "dog_bark"]
siren_data = df[df['class'] == "siren"]

car_horn_sampled = car_horn_data.sample(n=target_count, random_state=42)
dog_bark_sampled = dog_bark_data.sample(n=target_count, random_state=42)
siren_sampled = siren_data.sample(n=target_count, random_state=42)

# 클래스 1의 데이터와 샘플링한 클래스 3과 8의 데이터를 합치기
df1 = pd.concat([car_horn_sampled, dog_bark_sampled, siren_sampled])

In [41]:
def features_extract(file_name):
    sample, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    feature = librosa.feature.mfcc(y=sample, sr=sample_rate, n_mfcc=50)
    scaled_feature = np.mean(feature.T, axis=0)
    return scaled_feature

extracted = []
path = '/content/drive/MyDrive/'

for index_num, row in tqdm(df1.iterrows()):
    file_name = os.path.join(os.path.abspath(path), 'fold' + str(row["fold"]), str(row['slice_file_name']))
    final_class_labels = row['class']
    data = features_extract(file_name)
    extracted.append([data, final_class_labels])

0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(


In [42]:
ext_df = pd.DataFrame(extracted,columns=['feature','class'])
ext_df1=ext_df[ext_df['class'].isin(["car_horn","dog_bark","siren"])]
ext_df1

,feature,class
0,"[-346.73447, 117.378136, -46.578705, 33.662792...",car_horn
1,"[-317.48117, 155.55836, 4.206973, 52.135128, 1...",car_horn
2,"[-170.06764, 118.21796, -28.936142, 19.422232,...",car_horn
3,"[-490.05728, 96.37583, -28.977787, 34.217995, ...",car_horn
4,"[-316.254, 194.07347, -165.19958, 43.953053, -...",car_horn
...,...,...
1282,"[-225.59827, 165.52647, -55.945892, -4.5067225...",siren
1283,"[-220.92384, 166.31999, -13.943059, 23.526903,...",siren
1284,"[-199.61028, 174.74158, -66.42286, 6.6663556, ...",siren
1285,"[-599.5049, 215.67958, -15.634457, -0.04080167...",siren


In [43]:
x = np.array(ext_df1['feature'].tolist())
y = np.array(ext_df1['class'].tolist())

In [44]:
le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

In [45]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 42)

print("Number of training samples = ", x_train.shape[0])
print("Number of testing samples = ",x_test.shape[0])

Number of training samples =  1029
Number of testing samples =  258


In [46]:
num_labels = y.shape[1]
model = tf.keras.Sequential()

model.add(Dense(128, input_shape=(50,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128))

model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 128)               6528      
                                                                 
 activation_8 (Activation)   (None, 128)               0         
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 256)               33024     
                                                                 
 activation_9 (Activation)   (None, 256)               0         
                                                                 
 dropout_7 (Dropout)         (None, 256)               0         
                                                                 
 dense_12 (Dense)            (None, 256)              

In [47]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [48]:
model.fit(
          x_train,
          y_train,
          batch_size=32,
          epochs=200,
          validation_data=(x_test, y_test),
         )
#dropout을 높일수록 정확도 떨어짐

Epoch 1/200
33/33 [==============================] - 2s 16ms/step - loss: 19.7965 - accuracy: 0.3537 - val_loss: 1.1722 - val_accuracy: 0.7054
Epoch 2/200
33/33 [==============================] - 0s 7ms/step - loss: 7.6606 - accuracy: 0.4519 - val_loss: 1.1998 - val_accuracy: 0.6357
Epoch 3/200
33/33 [==============================] - 0s 8ms/step - loss: 4.2654 - accuracy: 0.4383 - val_loss: 0.7276 - val_accuracy: 0.6589
Epoch 4/200
33/33 [==============================] - 0s 8ms/step - loss: 2.7862 - accuracy: 0.4276 - val_loss: 0.7935 - val_accuracy: 0.6279
Epoch 5/200
33/33 [==============================] - 0s 10ms/step - loss: 1.8049 - accuracy: 0.4742 - val_loss: 0.8063 - val_accuracy: 0.7016
Epoch 6/200
33/33 [==============================] - 0s 6ms/step - loss: 1.6381 - accuracy: 0.4723 - val_loss: 0.8059 - val_accuracy: 0.7016
Epoch 7/200
33/33 [==============================] - 0s 6ms/step - loss: 1.3998 - accuracy: 0.4898 - val_loss: 0.8592 - val_accuracy: 0.5775
Epoch 8/20

In [49]:
model.save('outing_model', save_format="tf")

# 모델 로드
model = tf.keras.models.load_model('outing_model')
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 변환된 모델 저장
tflite_model = converter.convert()
tflite_model_path = '/content/drive/MyDrive/outing_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [50]:
def extract_feature(file_name):
    audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    fea = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=50)
    scaled = np.mean(fea.T,axis=0)
    return np.array([scaled])

In [51]:
def print_prediction(file_name):
    pred_fea = extract_feature(file_name)
    pred_vector = np.argmax(model.predict(pred_fea), axis=-1)
    pred_class = le.inverse_transform(pred_vector)
    print("The predicted class is:", pred_class[0], '\n')

In [52]:
print_prediction('/content/drive/MyDrive/fold1/24074-1-0-2.wav')
ipd.Audio('/content/drive/MyDrive/fold1/24074-1-0-2.wav')
#차소리

1/1 [==============================] - 0s 135ms/step
The predicted class is: car_horn 



In [53]:
print_prediction('/content/drive/MyDrive/fold1/7383-3-1-0.wav')
ipd.Audio('/content/drive/MyDrive/fold1/7383-3-1-0.wav')
#dog_bark

1/1 [==============================] - 0s 31ms/step
The predicted class is: dog_bark 



In [54]:
print_prediction('/content/drive/MyDrive/fold1/30823-8-0-0.wav')
ipd.Audio('/content/drive/MyDrive/fold1/30823-8-0-0.wav')
#siren

1/1 [==============================] - 0s 58ms/step
The predicted class is: siren 

